In [1]:
import requests
import lxml.html
import time
import pandas as pd

## requestsパッケージを用いて、滋賀大学DS学部の最新情報ページのHTMLを読み込む

In [2]:
url="https://www.ds.shiga-u.ac.jp/information/"
print("load {}".format(url))
time.sleep(5) # 多数のアクセスが殺到しないように、アクセス前にはtime.sleep()関数で少し待機しておく。
r = requests.get(url) # urlのHTMLを読み込み、rに代入
r.text # rに読み込まれたHTMLをテキストとして表示

load https://www.ds.shiga-u.ac.jp/information/


'<!DOCTYPE html>\r\n<html lang="ja">\r\n<head prefix="og: http://ogp.me/ns# website: http://ogp.me/ns/website#">\r\n<meta charset="UTF-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<!-- Google Tag Manager -->\r\n<script>\r\nwindow.dataLayer = window.dataLayer || [];\r\ndataLayer.push({\'ipAddress\': \'133.102.20.151\'});\r\n(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n})(window,document,\'script\',\'dataLayer\',\'GTM-N3R8SHPW\');\r\n</script>\r\n<!-- End Google Tag Manager -->\r\n<meta name="author" content="滋賀大学">\r\n<meta name="copyright" content="(C) 2023 Shiga University.">\r\n<!-- OGP -->\r\n<meta property="og:title" content="学部・研究科 | お知らせ | 滋賀大学 データサイエンス学部 / 研究科">\r\n<meta property="og:type" content="w

## lxmlパッケージで読み込んだHTMLをスクレイピング

In [3]:
html = lxml.html.fromstring(r.text) # テキストとしてのHTMLをlxmlパッケージで読み込み、パース
type(html) # パースされた結果は HtmlElementオブジェクトとして保存される

lxml.html.HtmlElement

In [4]:
# article要素を抜き出す
news_list = html.cssselect("article")
# news_list = html.cssselect("div.postBandWrap > article")
news_list

[<Element article at 0x2d229057220>,
 <Element article at 0x2d229086540>,
 <Element article at 0x2d229086770>,
 <Element article at 0x2d2290a83b0>,
 <Element article at 0x2d2290a86d0>,
 <Element article at 0x2d2290a8720>,
 <Element article at 0x2d2290a8770>,
 <Element article at 0x2d2290a87c0>,
 <Element article at 0x2d2290a8810>,
 <Element article at 0x2d2290a8860>,
 <Element article at 0x2d2290a88b0>,
 <Element article at 0x2d2290a8900>,
 <Element article at 0x2d2290a8950>,
 <Element article at 0x2d2290a89a0>,
 <Element article at 0x2d2290a89f0>,
 <Element article at 0x2d2290a8a40>]

In [5]:
news_item = news_list[0] # 最初のarticle要素をnews_itemとして取り出す
type(news_item) # 取り出された結果は再び HtmlElementオブジェクトになっている。

lxml.html.HtmlElement

In [6]:
# news_item内の各要素を取り出す
link_tags = news_item.cssselect("a")     
date_tags = news_item.cssselect("time")    
label_tags = news_item.cssselect("span")
text_tags = news_item.cssselect("h1")

In [7]:
# CSSセレクターの結果はリストなので[0]で最初の要素を取り出し、そこから、必要な情報を取り出す。
link = link_tags[0].attrib['href'] # URLはa要素のhref属性なので、.attribを用いて取り出す。
date = date_tags[0].text # .textはその要素の内容となるテキストを取り出す
label = label_tags[0].text
text = text_tags[0].text
print(link,date,label,text)

https://www.ds.shiga-u.ac.jp/information/1137/ 2024/06/03 お知らせ NVIDIA学生アンバサダーらが「デジタルツイン」に関するワークショップを実施


In [8]:
news_data=[]
for news_item in news_list:
    link_tags = news_item.cssselect("a")     
    date_tags=news_item.cssselect("time")    
    label_tags = news_item.cssselect("span")
    text_tags = news_item.cssselect("h1")
    
    link = link_tags[0].attrib['href']
    date = date_tags[0].text
    label = label_tags[0].text
    text = text_tags[0].text
    
    # 各 news_itemから得られたデータを辞書型のリストとして保存
    news_data.append({"date":date,"label":label,"text":text,"link":link})

In [9]:
df = pd.DataFrame(news_data) # 得られたデータをpandasのdataframeに変換
df

,date,label,text,link
0,2024/06/03,お知らせ,NVIDIA学生アンバサダーらが「デジタルツイン」に関するワークショップを実施,https://www.ds.shiga-u.ac.jp/information/1137/
1,2024/05/29,学部,竹村学長らが、灘中・高等学校でデータサイエンスについて講義,https://www.ds.shiga-u.ac.jp/information/1132/
2,2024/05/28,メディア掲載,データサイエンス学部の女子枠新設に関する記事が京都新聞に掲載,https://www.ds.shiga-u.ac.jp/information/1125/
3,2024/05/27,研究科,データサイエンス研究科の戸簾隼人さんが、ICBIR2024でBest Paper Awardを受賞,https://www.ds.shiga-u.ac.jp/information/1123/
4,2024/05/21,研究科,データサイエンス研究科交流会を開催,https://www.ds.shiga-u.ac.jp/information/1111/
5,2024/05/16,お知らせ,令和7年度（2025年度）滋賀大学データサイエンス学部・研究科 入学者選抜の変更点について（予告）,https://www.ds.shiga-u.ac.jp/information/1104/
6,2024/05/16,お知らせ,粉体工学会でデータサイエンス研究科修了生の北村智浩さんがベストプレゼンテーション賞を受賞,https://www.ds.shiga-u.ac.jp/information/1106/
7,2024/05/13,お知らせ,「世界で最も影響力のある研究者トップ2%」に本学研究者が選出,https://www.ds.shiga-u.ac.jp/information/1105/
8,2024/05/09,メディア掲載,【掲載】東洋経済オンラインにて滋賀大学の状況が紹介,https://www.ds.shiga-u.ac.jp/information/1099/
9,2024/05/09,メディア掲載,【掲載】竹村学長のインタビュー記事が「月刊 先端教育」6月号に掲載,https://www.ds.shiga-u.ac.jp/information/1103/


In [10]:
# 「次へ」に対応するURLを取得する
next_tags = html.cssselect("a.nextpostslink")
next_url = next_tags[0].attrib['href']
next_url

'https://www.ds.shiga-u.ac.jp/information/page/2/'

## これらを組み合わせて、クローラーを完成させる

In [11]:
next_url="https://www.ds.shiga-u.ac.jp/information/" # 最初のページ
news_data=[] # データを格納するリストの宣言/初期化

# 次のurlが見つかれば、そのページをスクレイピング
while next_url:
    # RequestsパッケージでHTMLを取得
    print("load {}".format(next_url))
    time.sleep(5) # 多数のアクセスが殺到しないように、アクセス前にはtime.sleep()関数で少し待機しておく。
    r = requests.get(next_url) 
    
    # lxmlでHTMLをパース
    html = lxml.html.fromstring(r.text)
    
    # 各ニュースに対応するarticle要素を抜き出す
    news_list = html.cssselect("article")
     
    # ニュースリスト内の各ニュースに対してループを回す。
    for news_item in news_list:
        # CSSセレクターで必要な要素を検索/取得
        link_tags = news_item.cssselect("a")     
        date_tags=news_item.cssselect("time")    
        label_tags = news_item.cssselect("span")
        text_tags = news_item.cssselect("h1")

        # 各要素から必要なデータを取得
        # やっていることは上のセルと同じだが、CSSセレクターでタグが見つからなかったときにエラーの代わりに、Noneが代入されるように微変更
        link = link_tags[0].attrib['href'] if link_tags else None
        date = date_tags[0].text if date_tags else None
        label = label_tags[0].text if label_tags else None
        text = text_tags[0].text if text_tags else None

        # 各 news_itemから得られたデータを辞書型のリストとして保存
        news_data.append({"date":date,"label":label,"text":text,"link":link})
    
    # 「次へ」に対応するURLを取得する
    next_tags = html.cssselect("a.nextpostslink")
    next_url = next_tags[0].attrib['href'] if next_tags else None

# 得られたデータをpandasのdataframeに変換
df = pd.DataFrame(news_data) 
df

load https://www.ds.shiga-u.ac.jp/information/
load https://www.ds.shiga-u.ac.jp/information/page/2/
load https://www.ds.shiga-u.ac.jp/information/page/3/
load https://www.ds.shiga-u.ac.jp/information/page/4/
load https://www.ds.shiga-u.ac.jp/information/page/5/
load https://www.ds.shiga-u.ac.jp/information/page/6/
load https://www.ds.shiga-u.ac.jp/information/page/7/
load https://www.ds.shiga-u.ac.jp/information/page/8/
load https://www.ds.shiga-u.ac.jp/information/page/9/
load https://www.ds.shiga-u.ac.jp/information/page/10/
load https://www.ds.shiga-u.ac.jp/information/page/11/
load https://www.ds.shiga-u.ac.jp/information/page/12/
load https://www.ds.shiga-u.ac.jp/information/page/13/
load https://www.ds.shiga-u.ac.jp/information/page/14/
load https://www.ds.shiga-u.ac.jp/information/page/15/
load https://www.ds.shiga-u.ac.jp/information/page/16/
load https://www.ds.shiga-u.ac.jp/information/page/17/
load https://www.ds.shiga-u.ac.jp/information/page/18/
load https://www.ds.shiga-

,date,label,text,link
0,2024/06/03,お知らせ,NVIDIA学生アンバサダーらが「デジタルツイン」に関するワークショップを実施,https://www.ds.shiga-u.ac.jp/information/1137/
1,2024/05/29,学部,竹村学長らが、灘中・高等学校でデータサイエンスについて講義,https://www.ds.shiga-u.ac.jp/information/1132/
2,2024/05/28,メディア掲載,データサイエンス学部の女子枠新設に関する記事が京都新聞に掲載,https://www.ds.shiga-u.ac.jp/information/1125/
3,2024/05/27,研究科,データサイエンス研究科の戸簾隼人さんが、ICBIR2024でBest Paper Awardを受賞,https://www.ds.shiga-u.ac.jp/information/1123/
4,2024/05/21,研究科,データサイエンス研究科交流会を開催,https://www.ds.shiga-u.ac.jp/information/1111/
...,...,...,...,...
318,2017/06/01,お知らせ,データサイエンス教育研究センター報【PDF】をアップしました,https://www.ds.shiga-u.ac.jp/information/347/
319,2016/12/22,お知らせ,データサイエンス教育強化を先導する拠点大学に選定,https://www.ds.shiga-u.ac.jp/information/346/
320,2016/10/06,お知らせ,データサイエンス学部のプロモーションビデオを公開しました(10/6),https://www.ds.shiga-u.ac.jp/information/345/
321,2016/08/06,お知らせ,和泉志津恵教授が公益財団法人大分県交通安全協会より功績賞を受賞(8月5日),https://www.ds.shiga-u.ac.jp/information/348/
